# ✨✨✨ VERSION 2.4 - CREATED December 17, 2025 ✨✨✨
# LangGraph Agent Learning - Healthcare Fraud Detection

**Purpose:** Learn LangGraph ReAct agents by wrapping existing UC Functions, Vector Search, and Genie API

**Domain:** Healthcare insurance claims (Upcoding, Phantom Billing, Provider Networks)

**Authentication:** Uses WorkspaceClient (same as dashboard) for realistic testing

**Branch:** `main` (production-ready demo)

---

## 🔄 **NOTEBOOK VERSION INFO**

**📅 Last Updated:** December 17, 2025
**🔧 Version:** 2.4 - WITH ARCHITECTURE DIAGRAMS
**✨ New in 2.1:** Visual architecture diagrams throughout notebook
**✅ Status:** Production-ready educational demo

**🚨 VERIFICATION:** After running LLM init cell, you MUST see:
```
✅ Tools bound to LLM (ensures proper JSON format)
```

**If you DON'T see this line → Detach & Reattach notebook!**

---

## What This Notebook Does

1. ✅ Tests all 4 tools individually (UC Functions, Vector Search, Genie)
2. ✅ Wraps them as LangChain Tools
3. ✅ Creates LangGraph ReAct Agent
4. ✅ Tests agent with different claim complexities (simple → suspicious → complex)
5. ✅ Compares Sequential vs Agent approaches (cost optimization)

**Goal:** Demonstrate how the fraud detection agent works under the hood

---

## 📊 Tutorial Architecture Overview

![5-Layer Architecture](../docs/images/technical_architecture_5layers.png)

*This tutorial teaches you to build the complete 5-layer architecture: from Databricks services (bottom) through LangChain tools, agents, ReAct pattern, up to LangGraph orchestration (top)*

**Note:** If images don't display in your Databricks workspace, see the [Image Setup Guide](../docs/images/README.md) for workspace deployment instructions.

---

## 🛠️ The Four Tools You'll Build

![Four Tool Architecture](../docs/images/4_tools_architecture.png)

*Each tool wraps a Databricks service, giving the agent different capabilities: classification, extraction, search, and analytics*

## 📦 Setup - Install Dependencies

**Why these specific versions?**
- `langgraph>=1.0.0` - Latest version (v1.0) removed `state_modifier` parameter
- `langchain>=0.3.0` - Latest version with improved tool calling
- Pinning versions ensures reproducibility across environments

**Key packages:**
- `langgraph` - State machine and agent orchestration
- `langchain` - Agent framework and tool abstractions
- `databricks-langchain` - Databricks-specific LangChain integrations
- `unitycatalog-langchain` - Unity Catalog function integration

In [ ]:
%pip install langgraph>=1.0.0 langchain>=0.3.0 langchain-core>=0.3.0 langchain-community>=0.3.0 databricks-langchain unitycatalog-langchain[databricks] backoff databricks-sdk mlflow databricks-vectorsearch --quiet

In [ ]:
dbutils.library.restartPython()

## ✅ Version Check - Run This First!

In [ ]:
# Version check - Run this to verify you have the latest notebook
import datetime
print("=" * 80)
print("🔍 NOTEBOOK VERSION CHECK")
print("=" * 80)
print("📅 Deployed Version: December 7, 2024 - 7:30 PM PST")
print("🔧 Version: 2.1 - WITH ARCHITECTURE DIAGRAMS")
print("=" * 80)
print("\n✅ If you see this, the notebook file is the latest version!")
print("🎨 NEW: Architecture diagrams embedded throughout notebook")
print("🚨 After running LLM init, you MUST see: 'Tools bound to LLM'")
print("❌ If you don't see that line → Detach & Reattach notebook")
print("=" * 80)

## 🔧 Imports and Configuration

**Import Strategy:**
- `WorkspaceClient` - Databricks SDK for calling APIs (UC Functions, Genie, etc.)
- `ChatDatabricks` - LangChain wrapper for Databricks Foundation Models
- `VectorSearchClient` - For semantic search in fraud knowledge base
- `SystemMessage` - LangChain message type for system prompts (v1.0+ pattern)
- `create_react_agent` - Pre-built ReAct agent from LangGraph

**Why these imports?**
- All use WorkspaceClient pattern (same as dashboard) for portability
- SystemMessage needed for manual prompt injection (v1.0+ requirement)
- No custom state classes needed for this simple use case

In [ ]:
# Core Databricks integrations
from databricks.sdk import WorkspaceClient  # Unified API client
from databricks_langchain import ChatDatabricks  # LLM wrapper
from databricks.vector_search.client import VectorSearchClient  # Semantic search

# LangChain/LangGraph components
from langchain_core.tools import Tool  # Tool wrapper abstraction
from langchain_core.messages import SystemMessage  # For system prompt injection
from langgraph.prebuilt import create_react_agent  # Pre-built ReAct pattern

# Additional imports for state management (if needed for advanced use)
from typing import Annotated
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages

# Standard library
import os
import json
import time
import backoff  # For retry logic

# Initialize WorkspaceClient (same pattern as dashboard)
w = WorkspaceClient()

print("✅ WorkspaceClient initialized")
print(f"📍 Workspace: {w.config.host}")
print(f"👤 Current user: {w.current_user.me().user_name}")

## ⚙️ Configuration from config.yaml

**Centralized Configuration Strategy:**
- Uses `shared/config.py` module (same as dashboard and all setup notebooks)
- Reads from `config.yaml` for single source of truth
- Auto-detects environment (dev/staging/prod) from DAB widget or env variable
- All settings (catalog, schema, warehouse, vector index, LLM) come from config

**Benefits:**
- ✅ No hardcoded values - change `config.yaml` once, works everywhere
- ✅ Environment-specific settings (dev vs prod)
- ✅ Consistent with dashboard and setup notebooks
- ✅ Easy to maintain and update

**LLM Endpoint:**
- Production uses Claude Sonnet 4 for reliable function calling
- Best JSON format compliance for tool calls
- Configured per environment in `config.yaml`

In [ ]:
# Import shared configuration module (same as dashboard and setup notebooks)
import sys
import os
sys.path.append(os.path.abspath('..'))
from shared.config import get_config

# Load configuration from config.yaml
# Auto-detects environment from DAB widget, env variable, or uses default
cfg = get_config()

# Extract config values for notebook use
CATALOG = cfg.catalog
SCHEMA = cfg.schema
WAREHOUSE_ID = cfg.warehouse_id
INDEX_NAME = cfg.vector_index  # Already computed as {catalog}.{schema}.fraud_cases_index
GENIE_SPACE_ID = cfg.genie_space_id
LLM_ENDPOINT = cfg.llm_endpoint

print("=" * 80)
print("✅ CONFIGURATION LOADED FROM config.yaml")
print("=" * 80)
print(f"🌍 Environment: {cfg.catalog.replace('fraud_detection_', '')}")
print(f"📚 Catalog: {CATALOG}")
print(f"📊 Schema: {SCHEMA}")
print(f"🏢 Warehouse ID: {WAREHOUSE_ID}")
print(f"🔍 Vector Index: {INDEX_NAME}")
print(f"🤖 Genie Space: {GENIE_SPACE_ID}")
print(f"🧠 LLM Endpoint: {LLM_ENDPOINT}")
print("=" * 80)
print("\n💡 TIP: Configuration is centralized in config.yaml")
print("   Change environment by passing to get_config('dev'/'prod')")
print("   Or set DAB widget 'environment' when deploying")
print("=" * 80)

---
# Part 1: Test Individual Tools

Before creating the agent, verify each tool works correctly

## 🔧 Tool 1: UC Function Wrapper (Classification & Extraction)

In [ ]:
def call_uc_function(function_name: str, parameters: dict):
    """
    Call a UC function using Statement Execution API (same as dashboard).
    This is more reliable for Databricks Apps and notebooks.
    """
    try:
        from databricks.sdk.service.sql import StatementState
        
        print(f"🔧 Calling UC Function: {CATALOG}.{SCHEMA}.{function_name}")
        
        # Build SQL query with parameters
        param_values = []
        for key, value in parameters.items():
            if isinstance(value, str):
                # Escape single quotes
                escaped = value.replace("'", "''")
                param_values.append(f"'{escaped}'")
            else:
                param_values.append(str(value))
        
        args_str = ', '.join(param_values)
        query = f"SELECT {CATALOG}.{SCHEMA}.{function_name}({args_str}) as result"
        
        print(f"  📝 SQL: {query[:100]}...")
        
        # Execute via Statement Execution API
        response = w.statement_execution.execute_statement(
            warehouse_id=WAREHOUSE_ID,
            statement=query,
            wait_timeout='30s'
        )
        
        if response.status.state == StatementState.SUCCEEDED:
            # Extract result from response
            if response.result and response.result.data_array:
                result_json = response.result.data_array[0][0]
                result = json.loads(result_json) if isinstance(result_json, str) else result_json
                print(f"✅ UC Function result received")
                return result
            else:
                print(f"❌ No data in response")
                return {"error": "No data returned"}
        else:
            error_msg = response.status.error.message if response.status.error else "Unknown error"
            print(f"❌ {error_msg}")
            return {"error": error_msg}
        
    except Exception as e:
        error_msg = f"Exception calling UC function: {str(e)}"
        print(f"❌ {error_msg}")
        import traceback
        traceback.print_exc()
        return {"error": error_msg}

In [ ]:
# Test 1: fraud_classify
print("=" * 80)
print("TEST 1: UC Function - fraud_classify")
print("=" * 80)

test_claim = "Database connection timeout in production affecting all users"
classification_result = call_uc_function(
    "fraud_classify",
    {"claim_text": test_claim}
)

print(f"\n📊 Classification Result:")
print(json.dumps(classification_result, indent=2))

In [ ]:
# Test 2: fraud_extract_indicators
print("=" * 80)
print("TEST 2: UC Function - fraud_extract_indicators")
print("=" * 80)

metadata_result = call_uc_function(
    "fraud_extract_indicators",
    {"claim_text": test_claim}
)

print(f"\n📊 Metadata Extraction Result:")
print(json.dumps(metadata_result, indent=2))

## 💬 Tool 3: UC Function - Fraud Explanation

In [ ]:
# Test 3: fraud_generate_explanation
print("=" * 80)
print("TEST 3: UC Function - fraud_generate_explanation")
print("=" * 80)

# Use results from previous tests
is_fraudulent = False  # Routine office visit is legitimate
fraud_type = "None"    # No fraud detected

explanation_result = call_uc_function(
    "fraud_generate_explanation",
    {
        "claim_text": test_claim,
        "is_fraudulent": is_fraudulent,
        "fraud_type": fraud_type
    }
)

print(f"\n💬 Fraud Explanation Result:")
print(json.dumps(explanation_result, indent=2))

## 🔍 Tool 4: Vector Search Wrapper

In [ ]:
def search_knowledge_base(query: str, num_results: int = 3):
    """
    Search fraud knowledge base using Vector Search.
    Same pattern as dashboard - directly portable!
    """
    try:
        print(f"🔍 Searching fraud knowledge base: '{query[:50]}...'")
        
        # Initialize VectorSearchClient (uses workspace client from environment)
        vsc = VectorSearchClient()
        
        results = vsc.get_index(
            index_name=INDEX_NAME
        ).similarity_search(
            query_text=query,
            columns=["doc_id", "doc_type", "title", "content"],
            num_results=num_results
        )
        
        docs = results.get('result', {}).get('data_array', [])
        print(f"✅ Found {len(docs)} documents")
        
        formatted_results = [
            {
                "doc_id": doc[0],
                "doc_type": doc[1],
                "title": doc[2],
                "content": doc[3][:200] + "..." if len(doc[3]) > 200 else doc[3],
                "score": doc[4] if len(doc) > 4 else None
            }
            for doc in docs
        ]
        
        return formatted_results
        
    except Exception as e:
        error_msg = f"Vector Search Error: {str(e)}"
        print(f"❌ {error_msg}")
        return []

In [ ]:
# Test 3: Vector Search
print("=" * 80)
print("TEST 3: Vector Search")
print("=" * 80)

search_query = "billing fraud patterns and upcoding schemes"
search_results = search_knowledge_base(search_query)

print(f"\n📚 Knowledge Base Results ({len(search_results)} documents):")
for i, doc in enumerate(search_results, 1):
    print(f"\n{i}. 📄 {doc['title']}")
    print(f"   📂 Type: {doc['doc_type']}")
    print(f"   📝 Content: {doc['content'][:150]}...")
    if doc.get('score'):
        print(f"   🎯 Score: {doc['score']:.4f}")

## 🤖 Tool 5: Genie Conversation API Wrapper

In [ ]:
class GenieConversationTool:
    """
    Genie Conversation API wrapper using WorkspaceClient.
    Same class as dashboard - directly portable!
    """
    def __init__(self, workspace_client: WorkspaceClient, space_id: str):
        self.w = workspace_client
        self.space_id = space_id
        print(f"[Genie] Initialized with space_id: {space_id}")
    
    def start_conversation(self, question: str):
        """Start a new Genie conversation"""
        try:
            print(f"[Genie] Starting conversation...")
            response = self.w.api_client.do(
                'POST',
                f'/api/2.0/genie/spaces/{self.space_id}/start-conversation',
                body={'content': question}
            )
            
            conversation_id = response.get('conversation_id')
            message_id = response.get('message_id')
            
            print(f"[Genie] ✅ Started - conversation_id: {conversation_id}, message_id: {message_id}")
            
            return {
                'status': 'started',
                'conversation_id': conversation_id,
                'message_id': message_id
            }
        except Exception as e:
            print(f"[Genie] ❌ Error starting conversation: {str(e)}")
            return {'status': 'error', 'error': str(e)}
    
    def poll_for_result(self, conversation_id: str, message_id: str, max_wait_seconds: int = 120):
        """Poll for query completion with exponential backoff"""
        print(f"[Genie] Polling for result (max {max_wait_seconds}s)...")
        start_time = time.time()
        poll_interval = 2
        poll_count = 0
        
        while time.time() - start_time < max_wait_seconds:
            try:
                poll_count += 1
                response = self.w.api_client.do(
                    'GET',
                    f'/api/2.0/genie/spaces/{self.space_id}/conversations/{conversation_id}/messages/{message_id}'
                )
                
                status = response.get('status')
                elapsed = time.time() - start_time
                print(f"[Genie] Poll #{poll_count} ({elapsed:.1f}s): Status = {status}")
                
                if status == 'COMPLETED':
                    print(f"[Genie] ✅ Query completed!")
                    return {'status': 'completed', 'response': response}
                elif status == 'FAILED':
                    error = response.get('error', {})
                    print(f"[Genie] ❌ Query failed: {error}")
                    return {'status': 'failed', 'response': response, 'error': error}
                
                time.sleep(poll_interval)
                poll_interval = min(poll_interval * 1.2, 10)
                
            except Exception as e:
                print(f"[Genie] ⚠️ Poll error: {str(e)}")
                time.sleep(poll_interval)
        
        print(f"[Genie] ⏱️ Timeout after {max_wait_seconds}s")
        return {'status': 'timeout', 'error': f'Query did not complete within {max_wait_seconds}s'}
    
    def query(self, question: str):
        """
        Complete Genie query workflow: start → poll → extract results → fetch data
        Returns structured results with actual data rows or error message.
        """
        print(f"\n[Genie] === Starting Genie Query ===")
        print(f"[Genie] Question: {question[:100]}...")
        
        # Step 1: Start conversation
        start_result = self.start_conversation(question)
        if start_result.get('status') != 'started':
            error_msg = f"Error starting conversation: {start_result.get('error')}"
            print(f"[Genie] {error_msg}")
            return {"error": error_msg}
        
        conversation_id = start_result['conversation_id']
        message_id = start_result['message_id']
        
        # Step 2: Poll for completion
        poll_result = self.poll_for_result(conversation_id, message_id, max_wait_seconds=120)
        
        poll_status = poll_result.get('status')
        
        if poll_status == 'failed':
            error_detail = poll_result.get('error', {}).get('message', 'Query execution failed')
            error_msg = f"Query failed: {error_detail}"
            print(f"[Genie] {error_msg}")
            return {"error": error_msg}
        
        if poll_status != 'completed':
            error_msg = f"Query did not complete: {poll_result.get('error', poll_status)}"
            print(f"[Genie] {error_msg}")
            return {"error": error_msg}
        
        # Step 3: Extract results from poll response
        response = poll_result['response']
        text_content = response.get('content', '')
        attachments = response.get('attachments', [])
        
        print(f"[Genie] Received response with {len(attachments)} attachments")
        
        result = {
            "text": text_content,
            "query": None,
            "data": None,
            "conversation_id": conversation_id,
            "message_id": message_id
        }
        
        # Step 4: Extract SQL query and attachment_id
        if attachments:
            attachment = attachments[0]
            
            # CRITICAL: Field is 'attachment_id', NOT 'id'!
            attachment_id = attachment.get('attachment_id') or attachment.get('id')
            print(f"[Genie] Attachment ID: {attachment_id}")
            
            # Extract SQL query
            query_obj = attachment.get('query', {})
            if isinstance(query_obj, dict):
                result['query'] = query_obj.get('query') or query_obj.get('content')
                if result['query']:
                    print(f"[Genie] Extracted SQL: {result['query'][:100]}...")
            
            # Step 5: Fetch actual data using query-result endpoint
            if attachment_id:
                print(f"[Genie] Calling query-result endpoint...")
                try:
                    query_result_response = self.w.api_client.do(
                        'GET',
                        f'/api/2.0/genie/spaces/{self.space_id}/conversations/{conversation_id}/messages/{message_id}/query-result/{attachment_id}'
                    )
                    
                    # Parse response - data is wrapped in 'statement_response'
                    statement_response = query_result_response.get('statement_response', {})
                    if statement_response:
                        print(f"[Genie] Found statement_response")
                        manifest = statement_response.get('manifest', {})
                        if manifest:
                            schema = manifest.get('schema', {})
                            columns = schema.get('columns', [])
                            column_names = [col.get('name') for col in columns]
                            print(f"[Genie] Found columns: {column_names}")
                            
                            # Get data rows
                            result_obj = statement_response.get('result', {})
                            data_array = result_obj.get('data_array', [])
                            print(f"[Genie] Found {len(data_array)} data rows from Genie API")
                            
                            if data_array:
                                # Convert to list of dicts
                                result['data'] = []
                                for row in data_array:
                                    row_dict = dict(zip(column_names, row))
                                    result['data'].append(row_dict)
                                print(f"[Genie] ✅ Successfully converted {len(result['data'])} rows")
                            else:
                                print(f"[Genie] No data_array in result")
                        else:
                            print(f"[Genie] No manifest in statement_response")
                    else:
                        print(f"[Genie] No statement_response in query result response")
                        
                except Exception as e:
                    print(f"[Genie] Error calling query-result endpoint: {str(e)}")
        
        # Step 6: FALLBACK - Execute SQL directly if we have query but no data
        if result.get('query') and not result.get('data'):
            print(f"[Genie] FALLBACK: No data from Genie API, executing SQL directly...")
            result['used_fallback'] = True
            try:
                from databricks.sdk.service.sql import StatementState
                
                execute_response = self.w.statement_execution.execute_statement(
                    warehouse_id=WAREHOUSE_ID,
                    statement=result['query'],
                    wait_timeout='30s'
                )
                
                print(f"[Genie] Fallback execution status: {execute_response.status.state}")
                
                if execute_response.status.state == StatementState.SUCCEEDED:
                    columns = execute_response.manifest.schema.columns if execute_response.manifest and execute_response.manifest.schema else []
                    column_names = [col.name for col in columns]
                    print(f"[Genie] Fallback found columns: {column_names}")
                    
                    if execute_response.result and execute_response.result.data_array:
                        data_array = execute_response.result.data_array
                        print(f"[Genie] Fallback found {len(data_array)} data rows")
                        
                        result['data'] = []
                        for row in data_array:
                            row_dict = dict(zip(column_names, row))
                            result['data'].append(row_dict)
                        print(f"[Genie] ✅ Fallback successfully converted {len(result['data'])} rows")
                else:
                    error_msg = execute_response.status.error.message if execute_response.status.error else "Unknown error"
                    print(f"[Genie] Fallback execution failed: {error_msg}")
            except Exception as e:
                print(f"[Genie] Fallback execution error: {str(e)}")
        
        print(f"[Genie] Final data rows extracted: {len(result.get('data') or [])}")
        
        # Determine which method was used
        if result.get('data'):
            if result.get('used_fallback'):
                result['method'] = 'Direct SQL Execution (Fallback)'
            else:
                result['method'] = 'Genie query-result API'
            print(f"[Genie] Method: {result['method']}")
        
        print(f"[Genie] === Genie Query Complete ===\n")
        return result

In [ ]:
# Test 4: Genie API
print("=" * 80)
print("TEST 5: Genie API")
print("=" * 80)

genie = GenieConversationTool(w, GENIE_SPACE_ID)
genie_question = "Show me the top 5 claims with the highest amounts"
genie_result = genie.query(genie_question)

print(f"\n🤖 Genie Result:")
print(f"  📝 Text Response: {genie_result.get('text', '')[:300]}...")
if genie_result.get('query'):
    print(f"  🔍 Generated SQL: {genie_result['query'][:200]}...")
if genie_result.get('data'):
    print(f"  📊 Data Rows: {len(genie_result['data'])}")
    print(f"  🔧 Method: {genie_result.get('method', 'Unknown')}")
    print(f"\n  Sample rows:")
    for i, row in enumerate(genie_result['data'][:3], 1):
        print(f"    {i}. {row}")
elif genie_result.get('error'):
    print(f"  ❌ Error: {genie_result['error']}")
else:
    print(f"  ⚠️ No data returned (query generated but data not fetched)")

---
# Part 2: Create LangChain Tools

Wrap the working functions as LangChain Tools for the agent

## 🧰 Wrap Functions as LangChain Tools

In [ ]:
print("=" * 80)
print("CREATING LANGCHAIN TOOLS")
print("=" * 80)

# Define Pydantic schemas for tool inputs (fixes LLM parameter naming issues)
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

class ClassifyClaimInput(BaseModel):
    claim_text: str = Field(description="The insurance claim text to classify for fraud")

class ExtractMetadataInput(BaseModel):
    claim_text: str = Field(description="The insurance claim text to extract fraud indicators from")

class SearchKnowledgeInput(BaseModel):
    query: str = Field(description="The search query to find relevant fraud patterns and documentation")

class ExplainFraudInput(BaseModel):
    claim_text: str = Field(description="The insurance claim text")
    is_fraudulent: str = Field(description="'true' if fraudulent, 'false' if legitimate")
    fraud_type: str = Field(description="Type of fraud detected (e.g., 'Upcoding', 'Phantom Billing')")

class QueryHistoricalInput(BaseModel):
    question: str = Field(description="Natural language question about historical fraud claims")

# Tool 1: Classification
def classify_claim_wrapper(claim_text: str) -> str:
    """
    Wrapper for fraud_classify UC Function.
    
    IMPORTANT: This function MUST be pure (no side effects)!
    - ❌ NO Streamlit calls (st.info, st.error, etc.) - causes NoSessionContext error
    - ✅ Return JSON string only
    - ✅ Include error handling in return value
    
    WHY: LangGraph runs tools in background threads without Streamlit session context
    """
    result = call_uc_function("fraud_classify", {"claim_text": claim_text})
    return json.dumps(result, indent=2) if result else json.dumps({"error": "Classification failed"})

classify_tool = Tool(
    name="classify_claim",
    description="Classifies an insurance claim as fraudulent or legitimate with fraud probability, type, and confidence score. Use this FIRST to assess fraud risk. Returns JSON with is_fraud, fraud_probability, fraud_type, confidence, key_indicators.",
    func=classify_claim_wrapper,
    args_schema=ClassifyClaimInput
)
print("✅ Tool 1: classify_claim")

# Tool 2: Metadata Extraction
def extract_metadata_wrapper(claim_text: str) -> str:
    """
    Wrapper for fraud_extract_indicators UC Function.
    IMPORTANT: Pure function - no Streamlit calls! (See classify_claim_wrapper for details)
    """
    result = call_uc_function("fraud_extract_indicators", {"claim_text": claim_text})
    return json.dumps(result, indent=2) if result else json.dumps({"error": "Extraction failed"})

extract_tool = Tool(
    name="extract_metadata",
    description="Extracts detailed fraud indicators from claims including red flags, suspicious patterns, affected entities, and risk scores. Use for suspicious or high-value claims to identify specific fraud signals. Returns JSON with red_flags, suspicious_patterns, affected_entities, risk_score.",
    func=extract_metadata_wrapper,
    args_schema=ExtractMetadataInput
)
print("✅ Tool 2: extract_metadata")

# Tool 3: Knowledge Base Search
def search_knowledge_wrapper(query: str) -> str:
    """
    Wrapper for Vector Search.
    IMPORTANT: Pure function - no Streamlit calls! (See classify_claim_wrapper for details)
    """
    try:
        results = search_knowledge_base(query, num_results=3)
        return json.dumps(results, indent=2) if results else json.dumps([])
    except Exception as e:
        return json.dumps({"error": f"Search failed: {str(e)}"})

search_tool = Tool(
    name="search_knowledge",
    description="Searches the fraud knowledge base for similar fraud cases, patterns, and schemes using semantic search. Use to find historical fraud cases with similar indicators or patterns. Returns JSON array with doc_id, title, content, doc_type for top matches.",
    func=search_knowledge_wrapper,
    args_schema=SearchKnowledgeInput
)
print("✅ Tool 3: search_knowledge")

# Tool 4: Fraud Explanation
def explain_fraud_wrapper(claim_text: str, is_fraudulent: str, fraud_type: str) -> str:
    """
    Wrapper for fraud_generate_explanation UC Function.
    IMPORTANT: Pure function - no Streamlit calls! (See classify_claim_wrapper for details)
    """
    # Convert string boolean to actual boolean for UC function
    is_fraud_bool = is_fraudulent.lower() == 'true'
    result = call_uc_function("fraud_generate_explanation", {
        "claim_text": claim_text,
        "is_fraudulent": is_fraud_bool,
        "fraud_type": fraud_type
    })
    return json.dumps(result, indent=2) if result else json.dumps({"error": "Explanation failed"})

explain_tool = StructuredTool(
    name="explain_fraud",
    description="Generates human-readable explanation for fraud decisions. Use AFTER classify and extract to explain WHY a claim is fraudulent or legitimate. Requires claim_text, is_fraudulent (true/false), and fraud_type. Returns JSON with explanation, evidence, and recommendations.",
    func=explain_fraud_wrapper,
    args_schema=ExplainFraudInput
)
print("✅ Tool 4: explain_fraud")



# Tool 5: Historical Trends (Genie API - Educational)
def query_historical_wrapper(question: str) -> str:
    """
    Wrapper for Genie Conversation API.
    NOTE: This is for educational purposes - shows how to integrate Genie as a tool.
    IMPORTANT: Pure function - no Streamlit calls! (See classify_claim_wrapper for details)
    """
    try:
        genie = GenieConversationTool(w, GENIE_SPACE_ID)
        result = genie.query(question)
        return json.dumps(result, indent=2, default=str)
    except Exception as e:
        return json.dumps({"error": f"Historical query failed: {str(e)}"})

genie_tool = Tool(
    name="query_historical",
    description="Queries historical fraud claims database using natural language to find patterns, statistics, and trends. Use for educational demonstration of Genie API integration. Returns JSON with text summary and optionally SQL query used.",
    func=query_historical_wrapper,
    args_schema=QueryHistoricalInput
)
print("✅ Tool 5: query_historical (Genie - Educational)")

print(f"\n🎉 All 5 LangChain Tools created successfully!")

---
# Part 3: Create LangGraph ReAct Agent

Build the agent that will intelligently decide which tools to use

## 🧠 Initialize LLM and Create Agent

**Critical LangGraph v1.0+ Pattern:**
This is the **main fix** that made the agent work!

**What Changed in LangGraph v1.0:**
- ❌ Old: `state_modifier` parameter (removed in v1.0)
- ✅ New: Pass SystemMessage in messages array at invocation time

**Why This Approach:**
1. **Compatible** - Works with LangGraph v1.0+
2. **Explicit** - Clear what messages are sent to LLM
3. **Flexible** - Can change system prompt per invocation if needed
4. **Simple** - No complex state management required

**Alternative Approaches (for reference):**
- LangGraph v0.2: Used `state_modifier` parameter
- LangGraph v1.0 advanced: Use custom `StateGraph` with preprocessor
- LangChain v1.0: Use `langchain.claims_analysis.create_agent()` with state_schema

We chose the simplest approach that works across versions!

In [ ]:
print("=" * 80)
print("CREATING LANGGRAPH REACT AGENT")
print("=" * 80)

# System prompt for the agent
# This guides the agent's decision-making process
# Key insight: Agent should be EFFICIENT, not exhaustive
system_prompt = """You are an expert healthcare fraud detection analyst.

Your goal is to efficiently analyze insurance claims for fraud - using only the tools 
that provide valuable fraud assessment information for each specific claim.

GUIDELINES:
1. ALWAYS classify the claim first to assess fraud risk and probability
2. For simple, low-value claims with low fraud probability, classification alone is often sufficient
3. For suspicious or high-value claims (>$10K), extract detailed fraud indicators
4. For claims with multiple red flags, search the fraud knowledge base for similar fraud patterns
5. For complex fraud schemes or provider pattern analysis, query historical fraud data
6. Be efficient but thorough - only use tools that add value to the fraud assessment

Think step-by-step, explain your reasoning, and make smart decisions about which tools to use.
"""

# Initialize LLM
# ChatDatabricks is a LangChain wrapper for Databricks Foundation Models
# It handles authentication via WorkspaceClient automatically
llm = ChatDatabricks(
    endpoint=LLM_ENDPOINT,  # Foundation model serving endpoint
    temperature=0.1,  # VERY LOW temp for reliable tool calling (was 0.3, reduced to 0.1)
    max_tokens=4096  # Sufficient for reasoning + tool calls
)
print(f"✅ LLM initialized: {LLM_ENDPOINT}")

# Create ReAct agent with all 4 tools
# ReAct = Reasoning + Acting (agent thinks before acting)
# Pattern: Thought → Action (tool call) → Observation (tool result) → repeat
tools_list = [classify_tool, extract_tool, search_tool, explain_tool, genie_tool]

# 🚨 CRITICAL FIX: Use bind_tools for reliable function calling
# This ensures the LLM uses proper JSON format (not XML) for ALL tool calls
# Without this, Claude might hallucinate XML format mid-conversation
llm_with_tools = llm.bind_tools(tools_list)
print(f"✅ Tools bound to LLM (ensures proper JSON format)")

# Create the agent using the standard create_react_agent
# IMPORTANT: Pass the bound LLM (llm_with_tools) not the raw LLM
# This ensures consistent tool calling format throughout the conversation
agent = create_react_agent(
    model=llm_with_tools,  # The LLM with tools bound (CRITICAL!)
    tools=tools_list  # Available tools the agent can call
    # NOTE: We do NOT pass state_modifier here (deprecated in v1.0)
    # Instead, we'll inject SystemMessage when we invoke the agent
)

print(f"✅ LangGraph ReAct Agent created!")
print(f"🧰 Agent has 5 tools available:")
print(f"   1. classify_claim")
print(f"   2. extract_metadata")
print(f"   3. search_knowledge")
print(f"   4. explain_fraud")
print(f"   5. query_historical (Genie - Educational)")
print(f"\n🎯 Agent will decide which tools to use based on the claim!")

---
# Part 4: Test Agent with Different Claims

See how the agent adapts to different claim complexities

## 🔄 How the ReAct Pattern Works

![ReAct Pattern Execution](../docs/images/flowchart_showing_react_claim.png)

*The agent loops through Reason → Act → Observe → Decide until it has enough information to provide a complete answer*

**Key Points:**
- Simple claims use 2-3 tools (fast, cheap)
- Complex claims use all 4 tools (thorough, accurate)
- Agent decides adaptively based on claim content

## 🧪 Test Runner Function

In [ ]:
def run_agent_test(claim_text: str, show_reasoning: bool = True):
    """
    Test agent with a claim and display results
    
    This function demonstrates the LangGraph v1.0+ invocation pattern
    
    **Key Pattern: Manual SystemMessage Injection**
    We pass the system prompt as the FIRST message when invoking the agent.
    This is the v1.0+ replacement for the deprecated state_modifier parameter.
    
    Args:
        claim_text: The support claim to analyze
        show_reasoning: Whether to display the agent's thought process
        
    Returns:
        Agent result containing messages and tool calls
    """
    print("\n" + "=" * 80)
    print(f"📋 CLAIM: {claim_text}")
    print("=" * 80 + "\n")
    
    start_time = time.time()
    
    # CRITICAL PATTERN: Manual SystemMessage Injection (v1.0+)
    # We explicitly pass SystemMessage as first message
    # This replaces the old state_modifier approach from v0.2
    result = agent.invoke({
        "messages": [
            # System message goes FIRST - sets agent behavior/personality
            SystemMessage(content=system_prompt),
            # Then user message with the actual task
            ("user", f"Analyze this insurance claim for potential fraud: {claim_text}")
        ]
    })
    # The agent will:
    # 1. Read system prompt (guidance)
    # 2. Read user message (task)
    # 3. Reason about what tools to call
    # 4. Call tools as needed
    # 5. Synthesize final answer
    
    elapsed_time = time.time() - start_time
    
    if show_reasoning:
        print("🧠 AGENT REASONING TRAIL:")
        print("-" * 80)
        
        for i, message in enumerate(result['messages']):
            msg_type = getattr(message, 'type', None)
            
            if msg_type == "human":
                print(f"\n👤 USER:")
                content = getattr(message, 'content', '')
                print(f"   {str(content)[:200]}...")
            elif msg_type == "ai":
                content = getattr(message, 'content', '') or ""
                # Check if this is a tool call or final answer
                tool_calls = getattr(message, 'tool_calls', None)
                if tool_calls:
                    print(f"\n🤖 AGENT THOUGHT & ACTION:")
                    if content:
                        print(f"   {str(content)[:300]}...")
                    for tool_call in tool_calls:
                        tool_name = tool_call.get('name') if isinstance(tool_call, dict) else getattr(tool_call, 'name', 'unknown')
                        tool_args = tool_call.get('args') if isinstance(tool_call, dict) else getattr(tool_call, 'args', {})
                        print(f"   🔧 Calling tool: {tool_name}")
                        print(f"   📥 Input: {str(tool_args)[:150]}...")
                else:
                    print(f"\n🤖 AGENT FINAL ANSWER:")
                    if content:
                        print(f"   {str(content)[:500]}...")
                    else:
                        print(f"   (No answer generated)")
            elif msg_type == "tool":  # Tool message
                tool_name = getattr(message, 'name', 'unknown')
                print(f"\n📤 TOOL RESULT ({tool_name}):")
                content = getattr(message, 'content', None)
                if content:
                    result_preview = str(content)[:200] if len(str(content)) > 200 else str(content)
                    print(f"   {result_preview}...")
                else:
                    print(f"   (No content)")
    
    # Count tools used
    tool_messages = [m for m in result['messages'] if getattr(m, 'type', None) == 'tool']
    tools_used = list(set([getattr(m, 'name', None) for m in tool_messages if getattr(m, 'name', None)]))
    
    print("\n" + "=" * 80)
    print(f"📊 SUMMARY:")
    print(f"   ⏱️  Time: {elapsed_time:.2f}s")
    if tools_used:
        print(f"   🧰 Tools used: {len(tools_used)}/4 - {', '.join(tools_used)}")
    else:
        print(f"   🧰 Tools used: 0/4")
    print("=" * 80 + "\n")
    
    return result

## 🧪 Test 1: Simple Legitimate Claim (Expected: 2 tools)

In [ ]:
print("🧪 TEST 1: SIMPLE LEGITIMATE CLAIM")
print("Expected behavior: Should use classify + search, skip extract and genie\n")

result_1 = run_agent_test("Routine office visit for annual checkup. Patient: Mary Johnson. Amount: $150. Provider: Community Health Clinic. Date: 3/20/2024.")

## 🧪 Test 2: High-Risk Fraud Claim (Expected: 4 tools)

In [ ]:
print("🧪 TEST 2: HIGH-RISK FRAUD CLAIM")
print("Expected behavior: Should use classify + extract + search + genie (all 4 tools)\n")

result_2 = run_agent_test("Emergency room visit for chest pain. Patient: Robert Wilson. Amount: $125,000. Provider: St. Mary's Hospital. Services include: cardiac catheterization, angioplasty, stent placement, ICU stay (5 days), multiple consultations. Billed to Medicare. Date: 3/10/2024.")

## 🧪 Test 3: Suspicious Fraud Pattern (Expected: 3 tools)

In [ ]:
print("🧪 TEST 3: SUSPICIOUS FRAUD PATTERN")
print("Expected behavior: Should use classify + extract + search (3 tools)\n")

result_3 = run_agent_test("Physical therapy sessions for back pain. Patient: Susan Martinez. Provider: Wellness PT Center. Frequency: 3x per week for 8 weeks (24 sessions total). Amount: $18,000. Same diagnosis code repeated across all sessions. Dates: 2/1/2024 - 3/31/2024.")

---
# Part 5: Sequential vs Agent Comparison

Compare the agent approach with the current sequential pipeline

## 🔄 Sequential Pipeline Function

In [ ]:
def sequential_pipeline(claim_text: str):
    """
    Run all 4 tools sequentially (current dashboard approach).
    Always executes all tools regardless of claim complexity.
    """
    print("\n" + "=" * 80)
    print("🔄 SEQUENTIAL PIPELINE - Running ALL tools")
    print("=" * 80 + "\n")
    
    start_time = time.time()
    results = {}
    
    # Step 1: Classify
    print("1️⃣ Step 1: Classifying claim...")
    results['classification'] = call_uc_function("fraud_classify", {"claim_text": claim_text})
    print(f"   ✅ Done\n")
    
    # Step 2: Extract
    print("2️⃣ Step 2: Extracting metadata...")
    results['metadata'] = call_uc_function("fraud_extract_indicators", {"claim_text": claim_text})
    print(f"   ✅ Done\n")
    
    # Step 3: Search
    print("3️⃣ Step 3: Searching fraud knowledge base...")
    results['knowledge'] = search_knowledge_base(claim_text)
    print(f"   ✅ Done\n")
    
    # Step 4: Genie
    print("4️⃣ Step 4: Querying historical claims via Genie...")
    genie = GenieConversationTool(w, GENIE_SPACE_ID)
    results['historical'] = genie.query(f"Find similar resolved claims about: {claim_text}")
    print(f"   ✅ Done\n")
    
    elapsed_time = time.time() - start_time
    
    print("=" * 80)
    print(f"📊 SEQUENTIAL SUMMARY:")
    print(f"   ⏱️  Time: {elapsed_time:.2f}s")
    print(f"   🧰 Tools used: 4/4 (always all tools)")
    print("=" * 80 + "\n")
    
    return results, elapsed_time

## 🆚 Side-by-Side Comparison

In [ ]:
print("\n" + "🆚" * 40)
print("COMPARISON TEST: Sequential vs Agent")
print("🆚" * 40 + "\n")

comparison_claim = "Prescription claim for diabetes medication. Patient: James Lee. Medication: Insulin glargine. Quantity: 90-day supply. Amount: $2,500. Pharmacy: QuickRx Pharmacy. Refilled 5 times in 3 months. Date: 3/25/2024."

print("📋 Testing with claim:")
print(f"   '{comparison_claim}'")
print("\n" + "=" * 80)

# Run Sequential
print("\n🅰️ APPROACH A: SEQUENTIAL PIPELINE (Current Dashboard)")
sequential_result, seq_time = sequential_pipeline(comparison_claim)

# Run Agent
print("\n🅱️ APPROACH B: LANGGRAPH AGENT (New Adaptive)")
agent_result = run_agent_test(comparison_claim, show_reasoning=True)

# Extract agent time and tools
agent_tools = [getattr(m, 'name', None) for m in agent_result['messages'] if getattr(m, 'type', None) == 'tool']
agent_tools_unique = list(set([t for t in agent_tools if t]))

print("\n" + "🆚" * 40)
print("COMPARISON RESULTS")
print("🆚" * 40 + "\n")

print("┌─────────────────────────┬──────────────────┬──────────────────┐")
print("│ Metric                  │ Sequential       │ Agent            │")
print("├─────────────────────────┼──────────────────┼──────────────────┤")
print(f"│ Tools Used              │ 4/4 (100%)       │ {len(agent_tools_unique)}/4 ({len(agent_tools_unique)*25}%)        │")
print(f"│ Efficiency              │ Fixed pipeline   │ Adaptive         │")
print(f"│ Predictability          │ High             │ Medium           │")
print(f"│ Flexibility             │ None             │ High             │")
print("└─────────────────────────┴──────────────────┴──────────────────┘")

print("\n💡 INSIGHTS:")
if len(agent_tools_unique) < 5:
    print(f"   ✅ Agent was more efficient: used {len(agent_tools_unique)}/5 tools vs 5/5")
    print(f"   ✅ Agent skipped: {set(['classify_claim', 'extract_metadata', 'search_knowledge', 'explain_fraud', 'query_historical']) - set(agent_tools_unique)}")
    print(f"   💰 Potential cost savings: ~{(5-len(agent_tools_unique))*20}%")
else:
    print(f"   ℹ️  Agent used all tools for this complex claim")
    print(f"   ℹ️  For simple claims, agent would be more efficient")

---
# ✅ Summary & Next Steps

## 🎉 Success Criteria Checklist

If you've successfully run all cells above, you have:

- ✅ **Tool 1 Working:** UC Function (fraud_classify) with WorkspaceClient
- ✅ **Tool 2 Working:** UC Function (fraud_extract_indicators) with WorkspaceClient
- ✅ **Tool 3 Working:** Vector Search with WorkspaceClient
- ✅ **Tool 4 Working:** Genie Conversation API with WorkspaceClient
- ✅ **All 4 LangChain Tools Created:** Successfully wrapped APIs
- ✅ **LangGraph ReAct Agent Created:** Agent with 4 tools and system prompt
- ✅ **Agent Makes Decisions:** Tested with different claim complexities
- ✅ **Comparison Complete:** Sequential vs Agent side-by-side
- ✅ **Code is Portable:** Everything uses WorkspaceClient (same as dashboard)

---

## 📊 Key Learnings

**What We Learned:**
1. **Agent adapts to claim complexity** - uses fewer tools for simple questions
2. **Tool descriptions matter** - they guide the agent's decision-making
3. **System prompt is critical** - sets the agent's behavior and efficiency
4. **ReAct pattern works** - Thought → Action → Observation loop is visible
5. **Trade-offs are real** - flexibility vs predictability

**When to Use Each Approach:**
- **Sequential:** Predictable, uniform claims; need consistency
- **Agent:** Varied complexity claims; want cost optimization

---

## 🚀 Next Steps

**Phase 2: Dashboard Integration**

Now that the agent works in the notebook, we can:

1. ✅ Extract working code to `dashboard/langraph_agent.py` module
2. ✅ Add 5th tab to dashboard: "🧪 LangGraph Agent (Experimental)"
3. ✅ Display agent reasoning trail in Streamlit UI
4. ✅ Add comparison mode in dashboard
5. ✅ Deploy and test in production

**Ready to move to dashboard integration!** 🎯

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║                                                                ║
║  🎉 LANGGRAPH AGENT NOTEBOOK COMPLETE! 🎉                     ║
║                                                                ║
║  ✅ All 4 tools tested and working                            ║
║  ✅ LangGraph ReAct Agent created                             ║
║  ✅ Agent makes intelligent decisions                         ║
║  ✅ Sequential vs Agent comparison done                       ║
║  ✅ Code is ready for dashboard integration                   ║
║                                                                ║
║  📍 Branch: agent_langraph_trying                             ║
║  📁 Notebook: notebooks/23_langraph_agent_learning.py         ║
║                                                                ║
║  🚀 Ready for Phase 2: Dashboard Integration!                 ║
║                                                                ║
╚════════════════════════════════════════════════════════════════╝
""")